```
sudo docker run --privileged --user carla --gpus all --net=host -e DISPLAY=$DISPLAY carlasim/carla-add:0.9.13 /bin/bash ./CarlaUE4.sh -world-port=12321 -RenderOffScreen
```

In [1]:
import carla

import json
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
from env.CarlaEnv import CarlaEnv
from utils.VideoRecorder import VideoRecorder

In [3]:
with open('./cfg/weather.json', 'r', encoding='utf8') as fff:
    weather_params = json.load(fff)
with open('./cfg/scenario.json', 'r', encoding='utf8') as fff:
    scenario_params = json.load(fff)

In [4]:
fps = 20

In [5]:
carla_env = CarlaEnv(
    weather_params=weather_params,
    scenario_params=scenario_params,
    selected_weather="L1",
    selected_scenario="jaywalk",
    carla_rpc_port=12321,
    carla_tm_port=18935,
    carla_timeout=8,
    perception_type="dvs_frame",
    num_cameras=5,
    rl_image_size=128,
    fov=60,
    max_fps=fps,
    min_fps=fps,
    max_episode_steps=1000,
    frame_skip=1,
)

carla env reset done.


In [6]:
throttles = []
brakes = []
velocitys = []

In [7]:
max_episode_num = 100
max_step_num = 200

In [8]:
video = VideoRecorder("./video", fps=fps)
video.init(True)

In [ ]:
for one_episode in range(max_episode_num):
    
    carla_env.reset()
    
    throttle = []
    brake = []
    velocity = []
    
    for one_step in range(max_step_num):
        
        if one_step <= 100:
            action = [0, 0.7]
        else:
            action = [0, -0.3]
        
        obs, reward, done, info = carla_env.step(action)

        video.record(obs, carla_env.vehicle)


        print(f"\rframe: {carla_env.frame}", end="")

        throttle.append(carla_env.vehicle.get_control().throttle)
        brake.append(carla_env.vehicle.get_control().brake)
        velocity.append(carla_env.vehicle.get_velocity().x)

    video.save(f"test-{one_episode+1}")
    print(f"\nsave video done.\n")

    throttles.append(throttle)
    brakes.append(brake)
    velocitys.append(velocity)
    

carla env reset done.
frame: 379
save video done.

carla env reset done.
frame: 251Collision (intensity 224.61797704275546)
Collision (intensity 51.81707098259146)
frame: 301Collision (intensity 156.61827593888847)
Collision (intensity 857.1324362700907)
Collision (intensity 213.62923188751998)
Collision (intensity 4.660282185560312)
Collision (intensity 28.732458828063596)
Collision (intensity 6.703749534446459)
frame: 302Collision (intensity 33.69836357581768)
Collision (intensity 9.745559035728363)
Collision (intensity 57.41039902813884)
Collision (intensity 30.66700224383797)
Collision (intensity 24.98985284080587)
Collision (intensity 38.220743588711876)
Collision (intensity 130.3106293731169)
Collision (intensity 10.454654998158407)
Collision (intensity 22.105186366005547)
Collision (intensity 13.33337232793836)
Collision (intensity 0.8388244863987504)
Collision (intensity 78.04961679542097)
Collision (intensity 16.14294490645484)
frame: 303Collision (intensity 69.86215591480989)

In [ ]:
times = list(range(len(velocity)))

plt.plot(times, velocity, label="velocity")
plt.plot(times, throttle, label="throttle")
plt.plot(times, brake, label="brake")
plt.legend()